In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from datetime import datetime
import pickle

In [3]:
trainFeaturesPath = '/content/drive/MyDrive/PS4/train-features.csv'
trainOutputPath = '/content/drive/MyDrive/PS4/train-output.csv'
testFeaturesPath = '/content/drive/MyDrive/PS4/test-features.csv'
finalPath = '/content/drive/MyDrive/PS4/submission.csv'


In [4]:
trainFeatures = pd.read_csv(trainFeaturesPath)
trainOutput = pd.read_csv(trainOutputPath)

In [5]:
trainFeatures.shape

(348607, 55)

In [6]:
trainFeatures = trainFeatures.drop('Id', axis=1)
trainFeatures = trainFeatures.drop('feature_28', axis = 1)


In [7]:
# trainOutput.head(5)

In [8]:
# trainFeatures.head(5)

In [9]:
numerical_features = ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9']

In [10]:
standardScaler = StandardScaler()

In [11]:
trainFeatures[numerical_features] = standardScaler.fit_transform(trainFeatures[numerical_features])

In [12]:
# X_train, X_test, y_train, y_test = train_test_split(trainFeatures, trainOutput['Covtype'].values, test_size=0.20, random_state=42)

In [13]:
# print(X_train.shape)
# print(type(y_train))
# print(X_test.shape)
# print(type(y_test))

In [14]:
# X_train.iloc[:, 0:10] = standardScaler.fit_transform(X_train.iloc[:, 0:10])
# X_test.iloc[:, 0:10] = standardScaler.transform(X_test.iloc[:, 0:10])

In [15]:
# X_train[numerical_features] = standardScaler.fit_transform(X_train[numerical_features])
# X_test[numerical_features] = standardScaler.transform(X_test[numerical_features])

In [16]:
# X_train.head(5)

In [17]:
# classifier = RandomForestClassifier(n_estimators = 69, criterion = 'entropy' , random_state = 0)
# classifier.fit(trainFeatures, trainOutput['Covtype'].values)

In [18]:
# test_df = pd.read_csv(testFeaturesPath)
# test_df = test_df.drop('feature_28', axis = 1)
# test_df[numerical_features] = standardScaler.transform(test_df[numerical_features])


In [19]:
# true_y_pred = classifier.predict(test_df.drop(columns=["Id"]))

In [20]:
# print(np.unique(true_y_pred))
# result_df = pd.DataFrame(list(zip(test_df.loc[:, "Id"], true_y_pred )), columns=["Id", "Covtype"])


In [21]:
# result_df.to_csv(finalPath, index=False)

In [22]:
# lr_list = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]

# for learning_rate in lr_list:
#     gb_clf = GradientBoostingClassifier(n_estimators=30, learning_rate=learning_rate, max_depth=3, random_state=0)
#     gb_clf.fit(X_train, y_train)

#     print("Learning rate: ", learning_rate)
#     print("Accuracy score (training): {0:.3f}".format(gb_clf.score(X_train, y_train)))
#     print("Accuracy score (validation): {0:.3f}".format(gb_clf.score(X_test, y_test)))

In [23]:
rf_estimator_classifiers = []

In [24]:
n_estimators_rf = [70,80,90]
n_estimators_et = [80,95]

In [25]:
oversampled_train_features = trainFeatures.append(trainFeatures)
oversampled_train_labels = np.concatenate((trainOutput['Covtype'].values, trainOutput['Covtype'].values),axis=0)

print(len(oversampled_train_features))
print(len(oversampled_train_labels))

697214
697214


In [26]:
start = datetime.now()

for n_estimator in n_estimators_rf:

  classifier = RandomForestClassifier(n_estimators=n_estimator, criterion='entropy', random_state=42)
  classifier.fit(oversampled_train_features, oversampled_train_labels)
  rf_estimator_classifiers.append(classifier)

print("Total time taken", datetime.now()-start)

Total time taken 0:08:56.219517


In [27]:
# paths = ["/content/drive/MyDrive/PS4/Models/RF_70.sav", "/content/drive/MyDrive/PS4/Models/RF_80.sav", "/content/drive/MyDrive/PS4/Models/RF_90.sav", "/content/drive/MyDrive/PS4/Models/ET_80.sav", "/content/drive/MyDrive/PS4/Models/ET_95.sav"]

In [28]:
# for path in paths:
#   rf_estimator_classifiers.append(pickle.load(open(path, 'rb')))

In [29]:
start = datetime.now()

for n_estimator in n_estimators_et:

  print("Training for Estimator size: ", n_estimator)
  classifier = ExtraTreesClassifier(n_estimators=n_estimator, criterion='entropy', max_depth=50, random_state=42)
  classifier.fit(oversampled_train_features, oversampled_train_labels)
  rf_estimator_classifiers.append(classifier)

print("Total time taken", datetime.now()-start)

Training for Estimator size:  80
Training for Estimator size:  95
Total time taken 0:03:49.441756


In [30]:
test_df = pd.read_csv(testFeaturesPath)
test_df = test_df.drop('feature_28', axis = 1)
test_df[numerical_features] = standardScaler.transform(test_df[numerical_features])
print(len(test_df))

232405


In [31]:
models_prediction_array = np.zeros((len(test_df),5), dtype=int)
for idx, model in enumerate(rf_estimator_classifiers):
  y_pred = model.predict(test_df.drop(columns=["Id"]))
  models_prediction_array[:,idx] = y_pred

In [32]:
print(models_prediction_array.shape)

(232405, 5)


In [33]:
final_predictions = []
for idx, row in enumerate(models_prediction_array):
  final_predictions.append(np.argmax(np.bincount(row)))

In [34]:
result_df = pd.DataFrame(list(zip(test_df.loc[:, "Id"], final_predictions )), columns=["Id", "Covtype"])
result_df.to_csv(finalPath, index=False)

In [35]:
paths = ["/content/drive/MyDrive/PS4/Models/RF_70.sav", "/content/drive/MyDrive/PS4/Models/RF_80.sav", "/content/drive/MyDrive/PS4/Models/RF_90.sav", "/content/drive/MyDrive/PS4/Models/ET_80.sav", "/content/drive/MyDrive/PS4/Models/ET_95.sav"]

In [36]:

for idx, classifier in enumerate(rf_estimator_classifiers):
  pickle.dump(classifier, open(paths[idx], "wb"))
